## Study of the 2D flow around a NACA wing shape

In [1]:
import lettuce as lt
import torch
import numpy as np
from scipy import interpolate
from matplotlib import pyplot as plt
from time import time

In [25]:
Ma = 0.15                   ## The speed of streaming

### APPLICATION ###
# turbine_diameter =
wing_length = 3             ## 'depth' of airfoil profile
tempC = 10                  ## degrees celcius
#p = 14.5                   ## air pressure
rho = 1.293                 ## kg/m³ air density
vchar = 15                  ## usually medium streaming velocity (may also be maximum velocity, around 1.5-times)
                            ## large wind turbines produce maximum power at 15 m/s. This can be assumed to be streaming velocity around the centre
dt_pu = 2e-5                ## this should allow up to 25,000 Hz

### DOMAIN ###
nx = 1400                   ## number of lattice nodes in x-direction
ny = 400                    ## number of lattice nodes in y-direction
shape = (nx, ny)            ## domain shape
x_wing_nose = 1             ## physical space before wing
x_wing_tail = 3             ## physical space behind wing
chord_length = wing_length  ## physical length of wing
domain_length_x = x_wing_nose + wing_length + x_wing_tail
dx = domain_length_x/nx     ## i.e. resolution
n_wing_nose = int(x_wing_nose//dx)  ## first grid point with wing
n_wing_tail = int(x_wing_tail//dx)  ## first grid point with wing

### FLOW CHARACTERISTICS ###
#Re = 5e6
lchar = wing_length         ## characteristic length in pu is obstacle length
temp = tempC+273.15         ## temperature in Kelvin
visc_dyn = 2.791e-7*temp**0.7355 ## dynamic viscosity of air
visc_kin = visc_dyn/rho     ## kinematic viscosity of air
Re = vchar*lchar/visc_kin   ## The type of streaming around the foil. Small (1.5m) 8e3,medium (3-5m) 2e5, large (>5m-150m) up to 5e6

# number of steps depends on Mach number and resolution (?)
#nmax = 100000
tmax = 10                   ## simulate tmax-seconds

### SIMULATION PARAMETERS ##
# how often to report (every n simulation steps)
nreport = 1000
# how often to print (every n simulation steps)
nconsole = 5000
# how often to plot (every n console steps)
nplot = 2
# test for convergence and crash
test_iterations = True
test_convergence = False
epsilon = 1e-7
# run pre-simulation with low Re to get rid of initialization pulses
Re_pre = 1000
n_pre = 2000  # wing profiles with camber may crash at low Re

### LETTUCE PARAMETERS ###
lattice = lt.Lattice(lt.D2Q9, torch.device("cuda:0"), use_native=False)


In [26]:
def mask_from_csv(x, y, wing_name):
    nx1, ny1 = np.shape(x)
    #n_wing_nose = int(nx1//5)    # wing starts after 1/5 of domain length
    #n_wing_tail = int(nx1*3//5)  # wing goes until 3/5 of domain length
    n_wing_height = int(ny1//2)  # wing sits at middle of domain length

    # read wing data from http://airfoiltools.com/plotter/index
    surface_data = np.genfromtxt(wing_name+'.csv', delimiter=",")[9:,:]
    surface_data = surface_data[:np.min(np.where(np.isnan(surface_data)[:,1])),:]
    zero_row = np.where(surface_data[:,0]==0)[0][0]
    x_data_top, y_data_top = surface_data[:zero_row,:].transpose()
    x_data_bottom, y_data_bottom = surface_data[zero_row:,:].transpose()

    # calculate scaling factor
    #x_wing_nose = x[n_wing_nose,0]
    #x_wing_tail = x[n_wing_tail,0]
    #available_length_x = x_wing_tail - x_wing_nose
    available_length_n = n_wing_tail-n_wing_nose
    actual_wing_length_x = max(max(x_data_top), max(x_data_bottom))
    scaling_factor = wing_length / actual_wing_length_x

    # scale wing data to fit domain restrictions
    x_data_top *= scaling_factor
    x_data_bottom *= scaling_factor
    y_data_top *= scaling_factor
    y_data_bottom *= scaling_factor

    # mapping data to the grid
    x_data_interp        = np.linspace(0, wing_length, available_length_n)            # [0 ... 5.05]
    y_data_top_interp    = interpolate.interp1d(x_data_top, y_data_top, fill_value="extrapolate")(x_data_interp)       # .interp1d object
    y_data_bottom_interp = interpolate.interp1d(x_data_bottom, y_data_bottom, fill_value="extrapolate")(x_data_interp) # .interp1d object

    # shifting the wing up by half the grid
    y_wing_height = y[0, n_wing_height]
    y_data_top_interp += y_wing_height
    y_data_bottom_interp += y_wing_height

    # setting y data in a 2D grid to compare with flow.grid[1]
    y_data_top_mapped = np.zeros(np.shape(y))
    y_data_top_mapped[n_wing_nose:n_wing_tail, :] = np.array([y_data_top_interp]).transpose()
    y_data_bottom_mapped = np.zeros(np.shape(y))
    y_data_bottom_mapped[n_wing_nose:n_wing_tail, :] = np.array([y_data_bottom_interp]).transpose()

    # creating mask
    bool_mask = (y < y_data_top_mapped) & (y > y_data_bottom_mapped)
    return bool_mask

In [27]:
class NacaWithPre(lt.Obstacle):
    def __init__(self, wing_name, filename=None):
        if filename is None:
            self.filename_base = r"/media/philipp/Storage/data/"+wing_name
        else:
            self.filename_base = r"/media/philipp/Storage/data/"+filename+'_pre'
        self.wing_name = wing_name
        self.n_pre = n_pre
        self.lattice = lattice
        super(NacaWithPre, self).__init__(shape, reynolds_number=Re, mach_number=Ma,lattice=lattice,domain_length_x=domain_length_x)
        x, y = self.grid
        self.mask = mask_from_csv(x, y, wing_name)
        self.pre_flow = lt.Obstacle(shape,reynolds_number=Re_pre,mach_number=Ma,lattice=lattice,domain_length_x=domain_length_x)
        self.pre_flow.mask = self.mask

    def initial_solution(self, x):
        # run a bit with low Re
        print('Doing ', n_pre, ' steps with Re = ', Re_pre, ' before actual run.')
        simulation = lt.Simulation(self.pre_flow, self.lattice, lt.RegularizedCollision(lattice, self.units.relaxation_parameter_lu),lt.StandardStreaming(lattice))
        print("total pre-time: ", self.pre_flow.units.convert_time_to_pu(self.n_pre))
        simulation.initialize_f_neq()
        simulation.reporters.append(lt.ObservableReporter(lt.IncompressibleKineticEnergy(lattice, self.pre_flow), interval=nconsole)) # print energy
        simulation.reporters.append(lt.VTKReporter(lattice, self.pre_flow, interval=nreport, filename_base=self.filename_base))
        simulation.step(self.n_pre)
        p = simulation.flow.units.convert_density_lu_to_pressure_pu(simulation.lattice.rho(simulation.f))
        u = self.pre_flow.units.convert_velocity_to_pu(lattice.u(simulation.f))
        return p, u

In [28]:
class Naca(lt.Obstacle):
    def __init__(self, wing_name):
        super(Naca, self).__init__(shape, reynolds_number=Re, mach_number=Ma,lattice=lattice,domain_length_x=domain_length_x,char_length=chord_length,char_velocity=vchar)
        x, y = self.grid
        self.mask = mask_from_csv(x, y, wing_name)

In [32]:
def setup_simulation(wing_name, filename=None):
    if filename is None:
        filename_base = r"/media/philipp/Storage/data/"+wing_name
    else:
        filename_base = r"/media/philipp/Storage/data/"+filename
    flow = Naca(wing_name)
    simulation = lt.Simulation(flow, lattice, lt.KBCCollision2D(lattice, flow.units.relaxation_parameter_lu),
                        lt.StandardStreaming(lattice))
    nmax = flow.units.convert_time_to_lu(tmax)
    print("Doing up to ", "{:.2e}".format(nmax), " steps.")
    print("Key paramters: Chord length", chord_length, "[m], Re", "{:.2e}".format(Re), "[1]")
    print("I will record every", nreport, "-th step, print every", nconsole, "-th step, and plot every", nconsole*nplot, "-th step.\n",
          "1000 steps correspond to", tmax/nmax*1e3, "seconds.\n")

    # set up reporters
    Energy = lt.IncompressibleKineticEnergy(lattice, flow)
    #energy_reporter_internal = lt.ObservableReporter(Energy, interval=nreport, out=None)
    #simulation.reporters.append(energy_reporter_internal)
    simulation.reporters.append(lt.ObservableReporter(Energy, interval=nconsole)) # print energy
    simulation.reporters.append(lt.VTKReporter(lattice, flow, interval=nreport, filename_base=filename_base))
    return simulation, flow, Energy, nmax

def run_n_plot(simulation, flow, Energy, nmax):
    # initialize simulation
    simulation.initialize_f_neq()
    if test_iterations:
        energy_new = 0
        mlups = 0
        iterations = int(nmax//nconsole)
        for i in range(iterations):
            mlups += simulation.step(nconsole)
            energy_old = energy_new
            energy_new = Energy(simulation.f).mean().item()
            rchange = abs((energy_new - energy_old)/energy_new)
            print("avg MLUPS: ", mlups/(i+1))
            if test_convergence and rchange < epsilon:
                print("CONVERGENCE! Less than ", epsilon*100, " % relative change")
                break
            elif test_convergence:
                print("no convergence, still ", round(rchange*100,5), " % relative change")
            elif not energy_new == energy_new:
                print("CRASHED!")
                break
            if i%nplot == 0:
                u = flow.units.convert_velocity_to_pu(lattice.u(simulation.f).detach().cpu().numpy())
                plt.imshow(u[0,...].T, origin="lower")
                plt.show()
        u = flow.units.convert_velocity_to_pu(lattice.u(simulation.f).detach().cpu().numpy())
        plt.imshow(u[0,...].T, origin="lower")
        plt.show()
    else:
        mlups = simulation.step(nmax)
        print("MLUPS: ", mlups)
        u = flow.units.convert_velocity_to_pu(lattice.u(simulation.f).detach().cpu().numpy())
        plt.imshow(u[0,...].T, origin="lower")
        plt.show()
    return

In [34]:
wing_dict = {
    'NACA-63215-highAOA',
    'NACA-63215-lowAOA',
    'NACA-63215-noAOA',
    'NACA-0012-noAOA'
}
setup = {}
for name in ['NACA-63215-highAOA']:
    filename = name+'_Re'+"{:.1e}".format(Re)
    setup[name] = setup_simulation(name, filename)

Doing up to  3.46e+05  steps.
Key paramters: Chord length 3 [m], Re 3.28e+06 [1]
I will record every 1000 -th step, print every 5000 -th step, and plot every 10000 -th step.
 1000 steps correspond to 0.028867513459481294 seconds.

steps     time     IncompressibleKineticEnergy


In [40]:
for name in ['NACA-63215-highAOA']:
    t = time()
    [simulation, flow, Energy, nmax] = setup[name]
    run_n_plot(simulation, flow, Energy, nmax)
    print(name, " took ", time()-t, " s\n")

0 0.0 0.0


KeyboardInterrupt: 